In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2
import open3d as o3d
from estimate_object_pose import BoxSegmentation, BoxPoseEstimation
from realsense_camera import RealsenseCamera
from camera_utils import stream
from copy import deepcopy
from time import time

def show(image):
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# initialization

camera = RealsenseCamera(enable_depth=True)

camera.start()
K = camera.get_K()
D = camera.get_D()
depth_scale = camera.get_depth_scale()
camera.stop()

box_segmentation = BoxSegmentation(erosion_size=10)
box_pose_estimation = BoxPoseEstimation(
    edges_sizes=np.array([0.11, 0.05, 0.022]), edge_points_per_cm=7,
    voxel_size=0.005, depth_scale=depth_scale, K=K, D=D,
    global_max_correspondence_distance=0.04, max_correspondence_distances=[0.02, 0.007])

In [ ]:
# test box segmentation

camera.start()
stream(camera)  # waits for a key to get image
image = camera.read()
camera.stop()

mask = box_segmentation.segment_box(image)
masked_image = image.copy()
masked_image[mask == 0] = 0
show(masked_image)

In [ ]:
# estimate box pose

camera.start()
stream(camera)  # waits for a key to get image
image, depth = camera.read(read_color=True, read_depth=True)
camera.stop()

mask = box_segmentation.segment_box(image)
box_pose = box_pose_estimation.estimate_pose(mask, depth)
if box_pose is None:
    print("Could not extract point cloud from depth image")

In [ ]:
# visualize point clouds

transformed_box_pc = deepcopy(box_pose_estimation.gt_pc)
transformed_box_pc.transform(box_pose)
transformed_box_pc.paint_uniform_color(np.array([1, 0, 0]))
extracted_box_pc = box_pose_estimation.extracted_pc
extracted_box_pc.paint_uniform_color(np.array([0, 1, 0]))
o3d.visualization.draw_geometries([transformed_box_pc, extracted_box_pc])

In [ ]:
# visualize box frame

force_z_axis_up = False
if force_z_axis_up:
    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        pose = np.matmul(box_pose, correction)
else:
    pose = box_pose

draw = image.copy()
rvec, _ = cv2.Rodrigues(pose[0:3, 0:3])
tvec = box_pose[0:3, 3]
cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.07)
show(draw)

In [ ]:
# stream box frame

def draw_box_frame(image, key, depth):
    tic = time()
    mask = box_segmentation.segment_box(image)
    box_pose = box_pose_estimation.estimate_pose(mask, depth)
    toc = time()
    print(toc - tic)

    if box_pose is None:
        return

    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        box_pose = np.matmul(box_pose, correction)

    rvec, _ = cv2.Rodrigues(box_pose[0:3, 0:3])
    tvec = box_pose[0:3, 3]
    cv2.drawFrameAxes(image, K, D, rvec, tvec, 0.07)

camera.start()
stream(camera, callbacks=draw_box_frame)
camera.stop()

debug section

In [ ]:
# debug box frame

max_n = 30
images = list()
depths = list()
draws = list()

def draw_box_frame(image, key, depth):
    images.append(image.copy())
    depths.append(depth.copy())

    tic = time()
    mask = box_segmentation.segment_box(image)
    box_pose = box_pose_estimation.estimate_pose(mask, depth)
    toc = time()
    print(toc - tic)

    if box_pose is None:
        draws.append(image.copy())
        if len(images) > max_n:
            images.pop(0)
            depths.pop(0)
            draws.pop(0)
        return

    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        box_pose = np.matmul(box_pose, correction)

    rvec, _ = cv2.Rodrigues(box_pose[0:3, 0:3])
    tvec = box_pose[0:3, 3]
    cv2.drawFrameAxes(image, K, D, rvec, tvec, 0.07)

    draws.append(image.copy())

    if len(images) > max_n:
        images.pop(0)
        depths.pop(0)
        draws.pop(0)

camera.start()
stream(camera, callbacks=draw_box_frame)
camera.stop()

In [ ]:
for i, (image, depth, draw) in enumerate(zip(images, depths, draws)):
    print(i)
    show(draw)

In [ ]:
i = 19
show(draws[i])
image = images[i]
depth = depths[i]